# 호텔 챗봇

이 노트북에서는 호텔 챗봇 문제를 다룹니다. 어떻게 하면 호텔 프론트 데스크에서 더 나은 고객 서비스를 제공할 수 있을까요? 간단한 방법 중 하나는 호텔에서 경험할 수 있는 일들에 관한 간단한 질문에 대답할 수 있는 FAQ 채팅 로봇을 갖는 것입니다. 챗봇을 갖는 것에는 많은 장점이 있습니다.

1. 호텔의 매력을 높이고 정보 처리량을 늘립니다.
2. 호텔에 대한 질문을 데이터 테이블 형식으로 수집할 수 있는 방법을 만듭니다.

이 노트북에서는 두 가지 모델을 살펴볼 것입니다. 코사인 유사성 및 doc2vec 모델입니다.


## 1. 챗봇에 지식 기반 추가

챗봇의 대화 능력은 사용할 수 있는 데이터에 의해 정의됩니다. Ques.txt 파일과 ans.txt 파일에서 질문과 답변을 살펴보십시오. 이 챗봇은 기본적으로 질문에 대하여 질문 은행과 코사인 유사성을 확인하여 답을 찾으려고 노력할 것입니다.

### 1.1 라이브러리 가져오기

In [1]:
import nltk # 텍스트 데이터를 처리
import numpy as np # 말뭉치를 배열로 표현
import random 
import operator
import string # 표준 파이썬 문자열을 처리
import re
from sklearn.metrics.pairwise import cosine_similarity # 이를 나중에 사용하여 두 개의 문장이 얼마나 비슷한지를 결정합니다.
from sklearn.feature_extraction.text import TfidfVectorizer # Experience 2에서 단어 가방을 만드는 함수를 만들었던 것을 기억하십니까? 이 함수는 같은 일을 합니다!

### 1.2 데이터 처리
#### 1.2.1 파일 읽기 및 저장

In [65]:
# 1.[Dataset] Module27 (ans).txt 파일을 읽어 raw_data_ans 에 저장한다.
# 2.[Dataset] Module27 (ques).txt 파일을 읽어 raw_data_ques 에 저장한다. 

# your code here

import os
# 단계 1: 파일 경로 지정하기읽기 및 저장 : ques_filepath, ans_filepath
ques_filepath = '[Dataset] cosmetics(ques)KO.txt'
ans_filepath = '[Dataset] cosmetics(ans)KO.txt'

# 질문 및 대답 파일 내용을 문자열로 읽어 소문자로 변경하여 저장하기 
with open(ques_filepath, 'r', encoding='utf-8') as file:
    raw_data_ques = file.read().lower()

with open(ans_filepath, 'r', encoding='utf-8') as file:
    raw_data_ans = file.read().lower()


# 저장된 데이터의 길이 확인
print(f'Length of raw_data_ques: {len(raw_data_ques)}')
print(f'Length of raw_data_ans: {len(raw_data_ans)}')

Length of raw_data_ques: 240
Length of raw_data_ans: 831


####  세분화, 표제어 추출,  단어 토큰화

모듈 26 코사인 유사성 노트북에서 배운 방법을 참조하여 데이터를 세분화, 토큰화, 표제어 추출로 전처리 진행합니다.

#### 1.2.3 문장 세분화 

In [66]:
# 1.2.3 문장 세분화 후 길이확인하기
# question 과 answer 데이터에 대해 sent_tokenize() 사용하여 문서를 문장 목록으로 변환한다 
# sent_tokens_ques / sent_tokens_ans  에 저장
# your code here


nltk.download('punkt')

sent_tokens_ques = nltk.sent_tokenize(raw_data_ques)
sent_tokens_ans = nltk.sent_tokenize(raw_data_ans)


# 토근화된 데이터 길이 확인하기 : sent_tokens_ans, sent_tokens_ques
print(f'Number of sentences in sent_tokens_ques: {len(sent_tokens_ques)}')
print(f'Number of sentences in sent_tokens_ans: {len(sent_tokens_ans)}')

Number of sentences in sent_tokens_ques: 10
Number of sentences in sent_tokens_ans: 21


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aruka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [67]:
# 질문과 답변 매칭한 딕셔너리 - ques_ans_pairs 생성한 후 상위 5개 출력해 보기
ques_ans_pairs = {sent_tokens_ques[i]: sent_tokens_ans[i] for i in range(min(len(sent_tokens_ques), len(sent_tokens_ans)))}

# 상위 5개 출력
for i, (ques, ans) in enumerate(ques_ans_pairs.items()):
    if i < 5:
        print(f'Q: {ques}')
        print(f'A: {ans}\n')
    else:
        break

Q: 저의 피부에 추천할 만한 제품을 알려주세요.
A: 고객님의 피부 타입에 따라 추천할 수 있는 제품이 다릅니다.

Q: 이 제품의 가격은 얼마인가요?
A: 피부 타입(지성, 건성 등)과 주요 문제(여드름, 주름 등)를 알려주시면 맞춤형 제품을 추천해 드리겠습니다.

Q: 이 제품의 주요 성분은 무엇인가요?
A: 제품의 가격은 제품 종류와 브랜드에 따라 다릅니다.

Q: 이 제품은 어떤 피부 타입에 적합한가요?
A: 구체적인 제품명을 알려주시면 정확한 가격을 확인해 드리겠습니다.

Q: 화장품을 사용하기 전에 무엇을 고려해야 하나요?
A: 제품의 주요 성분은 제품의 포장지나 웹사이트에서 확인할 수 있습니다.



#### 1.2.4 단어 토큰화

In [68]:
# raw_data_ques와 raw_data_ans 단어 토큰화하고 출력하기 : word_tokens_ques 저장
word_tokens_ques = nltk.word_tokenize(raw_data_ques)
word_tokens_ans = nltk.word_tokenize(raw_data_ans)

# 단어 토큰화된 결과 출력
print(f'Word tokens in questions: {word_tokens_ques[:50]}')  # 상위 50개 단어 출력
print(f'Word tokens in answers: {word_tokens_ans[:50]}')    # 상위 50개 단어 출력

Word tokens in questions: ['저의', '피부에', '추천할', '만한', '제품을', '알려주세요', '.', '이', '제품의', '가격은', '얼마인가요', '?', '이', '제품의', '주요', '성분은', '무엇인가요', '?', '이', '제품은', '어떤', '피부', '타입에', '적합한가요', '?', '화장품을', '사용하기', '전에', '무엇을', '고려해야', '하나요', '?', '이', '제품의', '사용법을', '알려주세요', '.', '이', '제품에', '대한', '고객', '리뷰를', '어디서', '볼', '수', '있나요', '?', '제품에', '알레르기', '반응이']
Word tokens in answers: ['고객님의', '피부', '타입에', '따라', '추천할', '수', '있는', '제품이', '다릅니다', '.', '피부', '타입', '(', '지성', ',', '건성', '등', ')', '과', '주요', '문제', '(', '여드름', ',', '주름', '등', ')', '를', '알려주시면', '맞춤형', '제품을', '추천해', '드리겠습니다', '.', '제품의', '가격은', '제품', '종류와', '브랜드에', '따라', '다릅니다', '.', '구체적인', '제품명을', '알려주시면', '정확한', '가격을', '확인해', '드리겠습니다', '.']


#### 1.2.5 표제어 추출
WordNetleMmatizer를 사용하여 단어 토큰에서 표제어를 추출하는 함수 작성

In [69]:
# LemTokens() 정의하기
from nltk.stem import WordNetLemmatizer

def LemTokens(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]

# 표제어 추출 결과 출력
lemmatized_tokens_ques = LemTokens(word_tokens_ques)
lemmatized_tokens_ans = LemTokens(word_tokens_ans)

print(f'Lemmatized tokens in questions: {lemmatized_tokens_ques[:50]}')  # 상위 50개 단어 출력
print(f'Lemmatized tokens in answers: {lemmatized_tokens_ans[:50]}')    # 상위 50개 단어 출력

Lemmatized tokens in questions: ['저의', '피부에', '추천할', '만한', '제품을', '알려주세요', '.', '이', '제품의', '가격은', '얼마인가요', '?', '이', '제품의', '주요', '성분은', '무엇인가요', '?', '이', '제품은', '어떤', '피부', '타입에', '적합한가요', '?', '화장품을', '사용하기', '전에', '무엇을', '고려해야', '하나요', '?', '이', '제품의', '사용법을', '알려주세요', '.', '이', '제품에', '대한', '고객', '리뷰를', '어디서', '볼', '수', '있나요', '?', '제품에', '알레르기', '반응이']
Lemmatized tokens in answers: ['고객님의', '피부', '타입에', '따라', '추천할', '수', '있는', '제품이', '다릅니다', '.', '피부', '타입', '(', '지성', ',', '건성', '등', ')', '과', '주요', '문제', '(', '여드름', ',', '주름', '등', ')', '를', '알려주시면', '맞춤형', '제품을', '추천해', '드리겠습니다', '.', '제품의', '가격은', '제품', '종류와', '브랜드에', '따라', '다릅니다', '.', '구체적인', '제품명을', '알려주시면', '정확한', '가격을', '확인해', '드리겠습니다', '.']


#### 1.2.6 정규화
지식 기반에 유용하지 않은 구두점을 제거하는 함수를 작성

In [70]:
# 1.2.6 정규화
# 지식 기반에 유용하지 않은 구두점을 제거하는 함수 작성

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

# LemNormalize() 정의하기
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# 정규화 결과 출력
normalized_tokens_ques = [LemNormalize(sentence) for sentence in sent_tokens_ques]
normalized_tokens_ans = [LemNormalize(sentence) for sentence in sent_tokens_ans]

print(f'Normalized tokens in questions: {normalized_tokens_ques[:2]}')  # 상위 2개 문장의 단어 출력
print(f'Normalized tokens in answers: {normalized_tokens_ans[:2]}')    # 상위 2개 문장의 단어 출력

Normalized tokens in questions: [['저의', '피부에', '추천할', '만한', '제품을', '알려주세요'], ['이', '제품의', '가격은', '얼마인가요']]
Normalized tokens in answers: [['고객님의', '피부', '타입에', '따라', '추천할', '수', '있는', '제품이', '다릅니다'], ['피부', '타입지성', '건성', '등과', '주요', '문제여드름', '주름', '등를', '알려주시면', '맞춤형', '제품을', '추천해', '드리겠습니다']]


## 2. 챗봇 기능 추가 - 코사인 유사성
데이터 세트를 문서 벡터로 변환 할 것입니다. 코사인 유사성은 유사한 벡터를 찾을 수 있게 해주며, 이러한 벡터는 의미가 비슷하다고 가정할 것입니다.

In [71]:
# 2.1.1 입력 및 응답 목록 작성
GREETING_INPUTS = ["hello", "hi", "greetings", "sup", "what's up","hey", "hey there"]
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

# 인사말을 수신하고 반환하는 함수 만들기
def greeting(sentence):
    for word in sentence.split(): # 문장의 각 단어를 살펴봅니다.
        if word.lower() in GREETING_INPUTS: # 단어가 GREETING_INPUT와 일치하는지 확인합니다.
            return random.choice(GREETING_RESPONSES) # Greeting_Response로 답장합니다.

In [72]:
# # 2.1.2 끝인삿말 대응하기
GOODBYE_INPUTS = ["bye", "see you!", "unit", "exit"]
GOODBYE_RESPONSES = ["Goodbye!", "See you later!", "Take care!", "Farewell!"]

# 끝인삿말을 수신하고 반환하는 함수 만들기
def goodbye(sentence):
    for word in sentence.split():
        if word.lower() in GOODBYE_INPUTS:
            return random.choice(GOODBYE_RESPONSES)
    return None

챗봇의 기능은 챗봇을 실행하기 위한 루프를 만듦으로써 이루어집니다. 아래 기능을 살펴봅니다. 함수의 각 줄은 중요한 단계를 수행하기 위해 다른 함수를 호출하기 때문에 중요합니다. 'response' 함수는 챗봇이 어떻게 행동하는지에 대한 작동 방식을 담당합니다.

In [73]:
# 2.1.3 질문을 받고 답변을 반환하는 함수 만들기
# your code here

def response(user_response):
    robo_response = ''
    sent_tokens_ques.append(user_response)
    tfidf_vectorizer = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(sent_tokens_ques)
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix)
    sim_scores = cosine_similarities.flatten()
    idx = sim_scores.argsort()[-2]
    flat = sim_scores.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    
    if req_tfidf == 0:
        robo_response = "I am sorry! I don't understand you."
    else:
        robo_response = sent_tokens_ans[idx]
    
    sent_tokens_ques.pop()
    return robo_response

마지막으로 챗봇 인터페이스를 만들고 이를 중심으로 페르소나를 만들어 봅시다. 우리가 챗봇을 '제인'이라고 부르고 코사인 유사성을 사용하여 질문과 유사한 FAQ를 찾아 대답하도록 합시다.

In [74]:
# 2.1.4 챗봇 테스트
# your code here

# 챗봇 테스트
print("Chatbot: 안녕하세요.무엇을 도와드릴까요? (Type 'bye' to exit)")

while True:
    user_input = input("You: ")
    if goodbye(user_input) is not None:
        print("Chatbot:", goodbye(user_input))
        break
    elif greeting(user_input) is not None:
        print("Chatbot:", greeting(user_input))
    else:
        print("Chatbot:", response(user_input))

Chatbot: 안녕하세요.무엇을 도와드릴까요? (Type 'bye' to exit)
You: 이 제품은 어떤 피부 타입에 적합한가요?


C:\Users\aruka\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\aruka\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


Chatbot: 구체적인 제품명을 알려주시면 정확한 가격을 확인해 드리겠습니다.
You: 이 제품의 사용법을 알려주세요.
Chatbot: 특정 제품에 대해 알고 싶으시면 제품명을 알려주세요.
You: 제품에 알레르기 반응이 있는지 어떻게 확인하나요?
Chatbot: 피부 타입에 따라 적합성을 확인하실 수 있습니다.
You: bye
Chatbot: Farewell!


## 3. Doc2vec를 이용한 챗봇 기능

챗봇을 만들기 위한 한 가지 유형의 모델을 더 다룰 것입니다. 코사인 유사성 모델 알고리즘은 두 문장 사이의 유사성을 찾는데 사용됩니다. 하지만 이제 신경망을 사용해서 이 문제를 해결해 보려합니다. 살펴보도록 하겠습니다.

Doc2Vec은 기본적으로 문서에서 벡터를 생성하는 신경망 기반 모델입니다. Doc2vec을 이해하기 위해서는 word2vec도 이해해야 합니다.

#### word2vec란?
이는 삽입 단어를 생성하는 모델이며, 여기서는 텍스트의 큰 말뭉치를 입력으로 받고 일반적으로 수백 개 차원의 벡터 공간을 생성합니다. 

2013년 9월과 10월 사이에 구글의 연구팀에 의해 두 개의 논문에 소개되었습니다. Word2Vec의 기본 가정은 유사한 맥락을 공유하는 두 단어가 유사한 의미를 공유하며 결과적으로 모델에서 유사한 벡터 표현을 공유한다는 것입니다.

예를 들어, "은행", "화폐", "계좌"는 "달러", "대출", "신용"과 같은 유사한 주변 단어와 함께 종종 사용되며, 따라서 Word2Vec에 따르면 이들은 유사한 벡터 표현을 공유합니다.

![Image](img1.png)

#### doc2vec 란?

doc2vec의 목적은 말뭉치의 모든 단어에 대한 특징 벡터를 계산하는 word2vec와 달리 문장/단락/문서의 수치 표현을 생성하는 것입니다. doc2vec은 말뭉치의 모든 문서에 대한 특징 벡터를 계산합니다. doc2vec에 의해 생성된 벡터는 문장/단락/문서 간의 유사성 찾기와 같은 작업에 사용될 수 있습니다.

<strong> doc2vec의 속성을 사용하여 우리만의 유사성 모델을 만들 것입니다.</strong>


관련 라이브러리를 가져오는 것으로 시작하겠습니다.

In [75]:
# subprocess와 sys 모듈을 사용하여 gensim 버전 3.8.1을 설치하는 부분입니다.
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "gensim == 3.8.1"])

0

In [76]:
# 추가로 gensim upgrade 실행하기
# !pip install --upgrade gensim

In [77]:
# gensim의 Doc2Vec와 TaggedDocument 클래스를 가져오고,
# nltk.tokenize에서 word_tokenize 함수를 가져옵니다.
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

첫 번째 주요 목표는 데이터에 태그를 지정하는 것입니다. doc2vec 모델은 데이터를 효과적으로 사용하기 위해 태그를 지정해야 합니다. 다음은 doc2vec에 대한 [시작 코드](https://www.kaggle.com/fmitchell259/creating-a-doc2vec-model) 좋은 학습 링크입니다. 다음 [링크](https://medium.com/wisio/a-gentle-introduction-to-doc2vec-db3e8c0cce5e) 도 유용하게 사용할 수 있으며 읽을 것을 권장합니다.

In [78]:
# 문장들을 소문자로 변환하고 단어로 토큰화하여 TaggedDocument 형식으로 변환한 후, tagged_data 리스트에 저장합니다.
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(sent_tokens_ques)]

In [79]:
# 최대 에포크 수를 100으로 설정합니다.
# 벡터의 크기를 20으로 설정합니다.
# 학습률을 0.025로 설정합니다.
max_epochs = 100
vec_size = 20 # 벡터가 더 크려면 이것을 증가시키세요. 이것은 더 많은 차이를 의미합니다. 
alpha = 0.025

다음 단계는 모델을 훈련시키는 것입니다. 이전처럼 훈련 과정을 실행하기 위해 `model.train` 함수를 사용하게 될 것입니다. Doc2vec 모델을 훈련하는 방법에 대한 자세한 정보는 [문서](https://radimrehurek.com/gensim/models/doc2vec.html) 를 참조하십시오.

In [80]:
# Doc2Vec 모델을 생성하고, 필요한 매개변수를 설정합니다.
model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm=1)

# model에 대한 단어 사전을 tagged_data를 사용하여 빌드합니다.
model.build_vocab(tagged_data)

# 주어진 최대 에포크 수에 대해 모델을 학습시킵니다.
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=1)
    model.alpha -= 0.0002
    model.min_alpha = model.alpha

# 학습된 모델을 "d2v.model"이라는 파일로 저장합니다. 
model.save("d2v.model")
print("Model Saved")


iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

### doc2vec 모델 평가

In [81]:
# 저장된 모델을 로드합니다.
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec.load("d2v.model")
model.save("doc2vec.bin")

In [82]:
# 테스트 데이터를 소문자로 변환하고 단어로 토큰화합니다.
test_data = word_tokenize("How much is the price?".lower())

`model.infer_vector` 함수를 사용하여 문서와 관련된 벡터를 추론할 수 있습니다. 그런 다음`most_similar` 함수를 사용하여 우리가 만든 벡터와 가장 유사한 벡터를 찾을 수 있습니다. 결과는 어떻습니까?

In [83]:
# 모델을 사용하여 문장의 벡터를 추론하고, v1에 저장한 후 출력합니다.
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

V1_infer [ 0.01741733 -0.01209098  0.0146148   0.00122488 -0.01746475 -0.02229971
 -0.02078249 -0.01272703  0.0048309   0.01675028 -0.01677033 -0.01475264
 -0.00651561  0.02311771 -0.01931787  0.01501887  0.0157516   0.0176538
  0.00582221 -0.00196055]


In [84]:
# v1과 가장 유사한 문서 4개를 찾아 출력합니다.
similar_doc = model.docvecs.most_similar(positive = [v1], topn = 4) #positive is an attribute that shows positive correlation first followed by the correlation value
print(similar_doc)

[('8', 0.17668059468269348), ('9', 0.13550099730491638), ('6', 0.09873324632644653), ('2', 0.05402598902583122)]


C:\Users\aruka\AppData\Local\Temp\ipykernel_228\3519407241.py:2: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar(positive = [v1], topn = 4) #positive is an attribute that shows positive correlation first followed by the correlation value


In [85]:
# 가장 유사한 문서의 인덱스를 얻고, 해당 인덱스에 해당하는 문서를 출력합니다.
num,_ = similar_doc[0]
num = int(num)
tagged_data[num]

TaggedDocument(words=['화장품의', '유통', '기한은', '어떻게', '확인하나요', '?'], tags=['8'])

이전 코드 블록의 출력에서 이 모델이 생각했던 것 만큼 효과적이지 않다는 것이 분명합니다. 이 모델이 성공하기를 기대했지만 실패한 이유가 있습니까?

자세한 내용을 보려면 이 [링크](https://stackoverflow.com/questions/58206571/doc2vec-find-the-similar-sentence) 를 클릭하십시오. 이 문제의 요지는 다음과 같습니다.

> Doc2Vec는 장난감 크기(toy-size)의 데이터셋에서 좋은 결과를 얻을 수 없으므로, 더 많은 데이터를 사용하기 전까지는 의미 있는 결과를 기대해서는 안 됩니다.

### 사전 학습된 doc2vec 사용

따라서 인터넷에서 다운로드한 사전 검증된 모델을 사용하여 사용 사례를 최적화해 보겠습니다. The 우리가 취득 한 모델은 관련 언론 뉴스에 대한 훈련을 받았으며  [여기](https://github.com/jhlau/doc2vec) 서 다운로드 할 수 있습니다.

모델을 로드하고 다시 평가해 보겠습니다.

In [86]:
# 현재 작업 디렉토리에서 "doc2vec.bin" 파일을 로드합니다.
model= Doc2Vec.load(os.getcwd()+r"//doc2vec.bin")

In [87]:
# 테스트 데이터를 소문자로 변환하고 단어로 토큰화합니다.
test_data = word_tokenize("How much is the price?".lower())

In [88]:
# 모델을 사용하여 문장의 벡터를 추론하고, v1에 저장한 후 출력합니다.
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

V1_infer [ 0.01741733 -0.01209098  0.0146148   0.00122488 -0.01746475 -0.02229971
 -0.02078249 -0.01272703  0.0048309   0.01675028 -0.01677033 -0.01475264
 -0.00651561  0.02311771 -0.01931787  0.01501887  0.0157516   0.0176538
  0.00582221 -0.00196055]


In [89]:
# sent_tokens에 있는 모든 문장에 대해 벡터를 추론하고, 각 문장과 v1 사이의 코사인 유사도를 출력합니다.
for i in sent_tokens_ques:
    v2 = model.infer_vector(word_tokenize(i.lower()))
    print(i)
    print(cosine_similarity(v1.reshape(1, -1),v2.reshape(1, -1)))

저의 피부에 추천할 만한 제품을 알려주세요.
[[-0.2525431]]
이 제품의 가격은 얼마인가요?
[[0.38808182]]
이 제품의 주요 성분은 무엇인가요?
[[0.04105826]]
이 제품은 어떤 피부 타입에 적합한가요?
[[-0.02006209]]
화장품을 사용하기 전에 무엇을 고려해야 하나요?
[[0.39660662]]
이 제품의 사용법을 알려주세요.
[[0.32489294]]
이 제품에 대한 고객 리뷰를 어디서 볼 수 있나요?
[[0.07771222]]
제품에 알레르기 반응이 있는지 어떻게 확인하나요?
[[0.19674292]]
화장품의 유통 기한은 어떻게 확인하나요?
[[0.14242297]]
이 제품을 어디에서 구매할 수 있나요?
[[-0.38730624]]


In [90]:
# 주어진 질문에 대한 답변을 계산하는 함수 calc_prob를 정의합니다.

def calc_prob(v1, q):
    probs = dict()
    for i in q:
        # 각 질문에 대해 벡터를 추론하고, v1과의 코사인 유사도를 계산합니다.
        v2 = model.infer_vector(word_tokenize(i.lower()))
        sim = cosine_similarity(v1.reshape(1, -1),v2.reshape(1, -1))
        #print(i)
        #print(sim)
        probs[i] = sim[0][0]
    
    sorted_d = dict( sorted(probs.items(), key=operator.itemgetter(1),reverse=True))
    
    # 유사도가 가장 높은 답변을 반환합니다.
    return list(sorted_d.items())[0]

In [91]:
calc_prob(v1, sent_tokens_ques)

('화장품을 사용하기 전에 무엇을 고려해야 하나요?', 0.39670742)

이 모델이 더 효과적인 것처럼 보이기 때문에, 우리의 챗봇에 통합해 봅시다.

In [93]:
# 사용자와 대화하는 부분입니다.

flag=True
print("제인: 제 이름은 제인입니다. 이 화장품에 대한 질문에 답변해 드리겠습니다. 나가려면 Bye를 입력하세요.")

while(flag==True):
    # 사용자의 입력을 받고, 소문자로 변환합니다.
    user_response = input()
    user_response=user_response.lower()
    # 사용자가 'bye'를 입력하기 전까지 다음 동작을 반복합니다:
    if(user_response!='bye'):
        # 사용자가 'thanks' 또는 'thank you'를 입력하면 대화를 종료하고 "You are welcome.."을 출력합니다.
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("제인: You are welcome..")
        else:
            # 사용자의 인사말에 대한 응답이 있는 경우 해당 응답을 출력합니다.
            if(greeting(user_response)!=None):
                print("제인: "+greeting(user_response))

            # 그렇지 않은 경우, 주어진 질문에 대한 답변을 계산하고 출력합니다.
            else:
                print("제인: ",end="")
                resp= calc_prob(model.infer_vector(word_tokenize(user_response)), sent_tokens_ques)
                print(ques_ans_pairs[resp[0]], )
                print(' (With similarity of ',resp[1],')')

    else:
        flag=False
        print("제인: 안녕히 가세요.")

제인: 제 이름은 제인입니다. 이 화장품에 대한 질문에 답변해 드리겠습니다. 나가려면 Bye를 입력하세요.
이 제품을 어디에서 구매할 수 있나요?
Jane: 이러한 정보를 바탕으로 적절한 제품을 선택하세요.
 (With similarity of  0.99997866 )
저의 피부에 추천할 만한 제품을 알려주세요.
Jane: 고객님의 피부 타입에 따라 추천할 수 있는 제품이 다릅니다.
 (With similarity of  0.99999785 )
화장품을 사용하기 전에 무엇을 고려해야 하나요?
Jane: 제품의 주요 성분은 제품의 포장지나 웹사이트에서 확인할 수 있습니다.
 (With similarity of  0.9998107 )
bye
제인: 안녕히 가세요.


**두 모델의 성능** 을 관찰한 후 다음과 같은 몇 가지 명확한 결론을 내릴 수 있습니다:

1. Doc2vec 모델은 단어 간의 관계를 이해하기 위해 더 많은 데이터가 필요합니다. 그리고 사전 훈련된 모델을 사용한 후에도 코사인 유사성 모델에 비해 모델의 응답의 품질이 아직 부족합니다.
2. 코사인 유사성 모델은 더 작고 잘 정의된 데이터 세트에서 더 잘 작동합니다. 이는 몇 가지 간단한 질문을 효과적으로 해결할 수 있지만 컨텍스트를 필요로 하는 복잡한 질문은 해결할 수 없다는 의미입니다.